<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Using AutoML for Predicting Sentence Similarity

This notebook demonstrates how to use Azure AutoML to automate machine learning model selection and tuning. It also demonstrates how to use a popular sentence embedding model from Google, Universal Sentence Encoder. 

### What is Azure AutoML?

Automated machine learning (AutoML) is a capability of Microsoft's Azure Machine Learning service. The goal of AutoML is to "improve the productivity of data scientists and democratize AI" [1] by allowing for the rapid development and deployment of machine learning models. To acheive this goal, AutoML automates the process of selecting a ML model and tuning the model. All the user is required to provide is a dataset (suitable for a classification, regression, or time-series forecasting problem) and a metric to optimize in choosing the model and hyperparameters. The user is also given the ability to set time and cost constraints for the model selection and tuning.

[1]https://azure.microsoft.com/en-us/blog/new-automated-machine-learning-capabilities-in-azure-machine-learning-service/

![](automl.png)

The AutoML model selection and tuning process can be easily tracked through the Azure portal or directly in python notebooks through the use of widgets. AutoML quickly selects a high quilty machine learning model tailored for your prediction problem. In this notebook, we walk through the steps of preparing data, setting up an AutoML experiment, and evaluating the results of our best model. More information about running AutoML experiments in Python can be found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train). 

### Modeling Problem

The regression problem we will demonstrate is predicting sentence similarity scores on the STS Benchmark dataset. The [STS Benchmark dataset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark#STS_benchmark_dataset_and_companion_dataset) contains a selection of English datasets that were used in Semantic Textual Similarity (STS) tasks 2012-2017. The dataset contains 8,628 sentence pairs with a human-labeled integer representing the sentences' similarity (ranging from 0, for no meaning overlap, to 5, meaning equivalence).

For each sentence in the sentence pair, we will use Google's pretrained Universal Sentence Encoder (details provided below) to generate a $512$-dimensional embedding. Both embeddings in the sentence pair will be concatenated and the resulting $1024$-dimensional vector will be used as features in our regression problem. Our target variable is the sentence similarity score.

In [1]:
# Set the environment path to find NLP
import sys
sys.path.append("../../")
import time
import os
import pandas as pd
import shutil
import numpy as np
import torch
import sys
from scipy.stats import pearsonr
from scipy.spatial import distance
from sklearn.externals import joblib

# Import utils
from utils_nlp.azureml import azureml_utils
from utils_nlp.dataset import stsbenchmark
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)

# Tensorflow dependencies for Google Universal Sentence Encoder
import tensorflow as tf
import tensorflow_hub as hub
tf.logging.set_verbosity(tf.logging.ERROR) # reduce logging output

# AzureML packages
import azureml as aml
import logging
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print("System version: {}".format(sys.version))
print("Azure ML SDK Version:", aml.core.VERSION)
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow Version:", tf.VERSION)

W0614 12:29:04.807609 38572 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Turning diagnostics collection on. 
System version: 3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)]
Azure ML SDK Version: 1.0.41
Pandas version: 0.23.4
Tensorflow Version: 1.13.1


In [2]:
BASE_DATA_PATH = '../../data'

# Data Preparation

## STS Benchmark Dataset

As described above, the STS Benchmark dataset contains 8.6K sentence pairs along with a human-annotated score for how similiar the two sentences are. We will load the training, development (validation), and test sets provided by STS Benchmark and preprocess the data (lowercase the text, drop irrelevant columns, and rename the remaining columns) using the utils contained in this repo. Each dataset will ultimately have three columns: _sentence1_ and _sentence2_ which contain the text of the sentences in the sentence pair, and _score_ which contains the human-annotated similarity score of the sentence pair.

In [3]:
# Load in the raw datasets as pandas dataframes
train_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")
dev_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="dev")
test_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="test")

100%|███████████████████████████████████████████████████████████████████████████████████████| 401/401 [00:01<00:00, 271KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|███████████████████████████████████████████████████████████████████████████████████████| 401/401 [00:01<00:00, 273KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|███████████████████████████████████████████████████████████████████████████████████████| 401/401 [00:01<00:00, 288KB/s]


Data downloaded to ../../data\raw\stsbenchmark


In [4]:
# Clean each dataset by lowercasing text, removing irrelevant columns,
# and renaming the remaining columns
train = stsbenchmark.clean_sts(train_raw)
dev = stsbenchmark.clean_sts(dev_raw)
test = stsbenchmark.clean_sts(test_raw)

In [5]:
print("Training set has {} sentences".format(len(train)))
print("Development set has {} sentences".format(len(dev)))
print("Testing set has {} sentences".format(len(test)))

Training set has 5749 sentences
Development set has 1500 sentences
Testing set has 1379 sentences


In [6]:
train.head(5)

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


## Feature Engineering:  Universal Sentence Encoder

Now that we have our sentence pairs loaded, we will convert these sentences into a numerical representation in order to use them in our machine learning model. To do this, we'll use a popular sentence encoder called Google Universal Sentence Encoder (see [original paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf)). Google provides two pretrained models based on different design goals: a Transformer model (targets high accuracy even if this reduces model complexity) and a Deep Averaging Network model (DAN; targets efficient inference). Both models are trained on a variety of web sources (Wikipedia, news, question-answers pages, and discussion forums) and produced 512-dimensional embeddings. This notebook utilizes the Transformer-based encoding model which can be downloaded [here](https://tfhub.dev/google/universal-sentence-encoder-large/3) because of its better performance relative to the DAN model on the STS Benchmark dataset (see Table 2 in Google Research's [paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf)). 

**Google Universal Sentence Encoder: Transformer Model** The Transformer model produces sentence embeddings using the "encoding sub-graph of the transformer architecture" (original architecture introduced [here](https://arxiv.org/abs/1706.03762)). "This sub-graph uses attention to compute context aware representations of words in a sentence that take into account both the ordering and identity of all the other workds. The context aware word representations are converted to a fixed length sentence encoding vector by computing the element-wise sum of the representations at each word position." The input to the model is lowercase PTB-tokenized strings and the model is designed to be useful for multiple different tasks by using multi-task learning. More details about the model can be found in the [paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf) by Google Research.

**Using the Pretrained Model**

Tensorflow-hub provides the pretrained model for use by the public. We import the model from its url and then feed the model our sentences for it to encode.

In [7]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"

# Import the Universal Sentence Encoder's TF Hub module
embedding_model = hub.Module(module_url)

In [8]:
def google_encoder(dataset):
    """ Function that embeds sentences using the Google Universal
    Sentence Encoder pretrained model
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    emb1: 512-dimensional representation of sentence1
    emb2: 512-dimensional representation of sentence2
    """
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    # Apply embedding model and normalize the input
    sts_encode1 = tf.nn.l2_normalize(embedding_model(sts_input1), axis=1)
    sts_encode2 = tf.nn.l2_normalize(embedding_model(sts_input2), axis=1)
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        emb1, emb2 = session.run(
          [sts_encode1, sts_encode2],
          feed_dict={
              sts_input1: dataset['sentence1'],
              sts_input2: dataset['sentence2']
          })
    return emb1, emb2

As features, we will embed both sentences using the Google Universal Sentence Encoder and concatenate their representations into a $1024$-dimensional vector. The resulting data will be saved in a dataframe for consumption by our AutoML model.

In [9]:
def feature_engineering(dataset):
    """Extracts embedding features from the dataset and returns
    features and target in a dataframe
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    df: pandas dataframe with embedding features and target variable
    """
    google_USE_emb1, google_USE_emb2 = google_encoder(dataset)
    n_google = google_USE_emb1.shape[1] #length of the embeddings 
    df = np.concatenate((google_USE_emb1, google_USE_emb2), axis=1)
    names = ['USEEmb1_'+str(i) for i in range(n_google)]+['USEEmb2_'+str(i) for i in range(n_google)]
    df = pd.DataFrame(df, columns=names)
    df['score'] = dataset['score'].tolist()
    return df

In [10]:
training_data = feature_engineering(train)
validation_data = feature_engineering(dev)
testing_data = feature_engineering(test)

In [60]:
#Take this out later
training_data.to_csv(os.path.join(featurized_data_location,"googleUSE_features_train.csv"), index=None)
testing_data.to_csv(os.path.join(featurized_data_location,"googleUSE_features_test.csv"), index=None)
validation_data.to_csv(os.path.join(featurized_data_location,"googleUSE_features_dev.csv"), index=None)

# Calculate Baseline Performance

Before using AutoML we will calculate a baseline to compare the AutoML results to. For the baseline we will take the Google Universal Sentence Encoder embeddings of each sentence, calculate the cosine similarity between the two sentence embeddings, then compare the predicted values with the true scores using pearson correlation. 

### What is Pearson Correlation?

Our evaluation metric is Pearson correlation ($\rho$) which is a measure of the linear correlation between two variables. The formula for calculating Pearson correlation is as follows:  

$$\rho_{X,Y} = \frac{E[(X-\mu_X)(Y-\mu_Y)]}{\sigma_X \sigma_Y}$$

This metric takes a value in [-1,1] where -1 represents a perfect negative correlation, 1 represents a perfect positive correlation, and 0 represents no correlation. We utilize the Pearson correlation metric as this is the metric that [SentEval](http://nlpprogress.com/english/semantic_textual_similarity.html), a widely-used evaluation toolkit for evaluation sentence representations, uses for the STS Benchmark dataset.

In [11]:
def get_baseline_performance(data):
    """ Get baseline performance by calculating the cosine similarity between
    the embeddings in the sentence pair and then evaluating the pearson 
    correlation between the predicted and true similarity scores
    
    Parameters:
    ----------
    data: dataframe containing embeddings and similarity scores
    """
    emb1 = data[[i for i in data.columns if 'USEEmb1' in i]].values.tolist()
    emb2 = data[[i for i in data.columns if 'USEEmb2' in i]].values.tolist()
    scores = data['score'].values.tolist()
    
    predictions = [1-distance.cosine(emb1[i], emb2[i]) for i in range(len(emb1))]
    print("Google Universal Sentence Encoder Pearson Correlation:", round(pearsonr(predictions, scores)[0],3))

In [12]:
get_baseline_performance(testing_data)

Google Universal Sentence Encoder Pearson Correlation: 0.764


# AutoML

AutoML can be used for classification, regression or timeseries experiments. Each experiment type has corresponding machine learning models and metrics that can be optimized (see [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train)) and the options will be delineated below. As a first step we connect to an existing workspace or create one if it doesn't exist.

In [13]:
ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>"
)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

W0614 12:37:29.502033 38572 authentication.py:494] Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Performing interactive authentication. Please follow the instructions on the terminal.


W0614 12:37:29.827009 11124 _profile.py:1082] Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
W0614 12:37:37.015321 38572 _profile.py:774] You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.
Workspace name: MAIDAPTest
Azure region: eastus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: nlprg


## AutoMLConfig Parameters
Next, we specify the parameters for the AutoMLConfig class. 

**task**  
AutoML supports the following base learners for the regression task: Elastic Net, Light GBM, Gradient Boosting, Decision Tree, K-nearest Neighbors, LARS Lasso, Stochastic Gradient Descent, Random Forest, Extremely Randomized Trees, XGBoost, DNN Regressor, Linear Regression. In addition, AutoML also supports two kinds of ensemble methods: voting (weighted average of the output of multiple base learners) and stacking (training a second "metalearner" which uses the base algorithms' predictions to predict the target variable). Specific base learners can be included or excluded in the parameters for the AutoMLConfig class (whitelist_models and blacklist_models) and the voting/stacking ensemble options can be specified as well (enable_voting_ensemble and enable_stack_ensemble)

**preprocess**  
AutoML also has advanced preprocessing methods, eliminating the need for users to perform this manually. Data is automatically scaled and normalized but an additional parameter in the AutoMLConfig class enables the use of more advanced techniques including imputation, generating additional features, transformations, word embeddings, etc. (full list found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-portal-experiments#preprocess)). Note that algorithm-specific preprocessing will be applied even if preprocess=False. 

**primary_metric**  
The regression metrics available are the following: Spearman Correlation (spearman_correlation), Normalized RMSE (normalized_root_mean_squared_error), Normalized MAE (normalized_mean_absolute_error), and R2 score (r2_score) 

**Constraints:**  
There is a cost_mode parameter to set cost prediction modes (see options [here](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlconfig?view=azure-ml-py)). To set constraints on time there are multiple parameters including experiment_exit_score (target score to exit the experiment after acheiving), experiment_timeout_minutes (maximum amount of time for all combined iterations), and iterations (total number of different algorithm and parameter combinations to try).

In [14]:
automl_settings = {
    "task": 'regression', #type of task: classification, regression or forecasting
    "debug_log": 'automated_ml_errors.log',
    "path": './automated-ml-regression',
    "iteration_timeout_minutes" : 15, #How long each iteration can take before moving on
    "iterations" : 50, #Number of algorithm options to try
    "primary_metric" : 'spearman_correlation', #Metric to optimize
    "preprocess" : True, #Whether dataset preprocessing should be applied
    "verbosity":logging.ERROR}

In [15]:
X_train = training_data.drop("score", axis=1).values
y_train = training_data['score'].values.flatten()
X_validation = validation_data.drop("score", axis=1).values
y_validation = validation_data['score'].values.flatten()

# local compute
automated_ml_config = AutoMLConfig(
     X = X_train,
     y = y_train,
     X_valid = X_validation,
     y_valid = y_validation,
     **automl_settings)

## Run the Experiment

Run the experiment locally and inspect the results using a widget

In [16]:
experiment=Experiment(ws, 'automated-ml-regression')
local_run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_91012708-fd8f-405f-afed-4c42f8fe3fc6
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0 

The results of the completed run can be visualized in two ways. First, by using a RunDetails widget as shown in the cell below. Second, my accessing the [Azure portal](https://portal.azure.com), selecting your workspace, clicking on _Experiments_ and then selecting the name and run number of the experiment you want to inspect. Both these methods will show the results and duration for each iteration (algorithm tried), a visualization of the results, and information about the run including the compute target, primary metric, etc.

In [17]:
# Inspect the run details using the provided widget
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

## Inspect the Best Model

Now we can identify the model that maximized performance on a given metric (spearman correlation in our case). The object returned by AutoML is a Pipeline class which chains together multiple steps in a machine learning workflow in order to provide a "reproducible mechanism for building, evaluating, deploying, and running ML systems" (see [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-getting-started.ipynb) for additional information about Pipelines). Our best model is a Pipeline with two steps: a DataTransformer step and a PreFittedSoftVotingRegressor step. We demonstrate how to extract additional information about what data transformations were used and which models make up the ensemble.

In [18]:
lookup_metric = "spearman_correlation"
best_run, fitted_model = local_run.get_output(metric = lookup_metric)
print(fitted_model)

RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_feature_sweeping=None, feature_sweeping_timeout=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('stackensembleregressor', StackEnsembleRegressor(base_learners=[('28', Pipeline(memory=None,
     steps=[('standardscaler...   random_state=None, selection='cyclic', tol=0.0001, verbose=0),
            training_cv_folds=5))]),
          stddev=None)


We can look at the different models that are used to produce the stack ensemble model

In [29]:
fitted_model.named_steps['stackensembleregressor'].get_params()

{'base_learners': None,
 'meta_learner': None,
 'training_cv_folds': None,
 '28': Pipeline(memory=None,
      steps=[('standardscalerwrapper', <automl.client.core.common.model_wrappers.StandardScalerWrapper object at 0x0000025C12972C88>), ('xgboostregressor', XGBoostRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bytree=1, eta=0.01, gamma=0, learning_rate=0.1,
          ma...ale_pos_weight=1, seed=None,
          silent=True, subsample=0.7, tree_method='auto', verbose=-10))]),
 '44': Pipeline(memory=None,
      steps=[('sparsenormalizer', <automl.client.core.common.model_wrappers.SparseNormalizer object at 0x0000025C129917F0>), ('xgboostregressor', XGBoostRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bytree=1, eta=0.1, gamma=0, grow_policy='lossguide',
          learnin...scale_pos_weight=1, seed=None, silent=True, subsample=1,
          tree_method='hist', verbose=-10))]),
 '30': Pipeline(memory=None,
      st

We can also look at how each column in our dataset was featurized by AutoML

In [37]:
fitted_model.named_steps['datatransformer'].get_featurization_summary()

[{'RawFeatureName': 'C1',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'C2',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'C3',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'C4',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'C5',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'C6',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'C7',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},

## Prediction
Finally, we can use the best model to make a prediction on our test set using pearson correlation as our metric

In [36]:
X_test = testing_data.drop("score", axis=1).values
y_test = testing_data['score'].values.flatten()

y_pred = fitted_model.predict(X_test)
print(pearsonr(y_pred, y_test)[0])

0.7722744639946534


In [44]:
#Save the model as a pkl file
model_path = 'sentence_similarity_regressor.pkl'
joblib.dump(fitted_model, model_path)

['sentence_similarity_regressor.pkl']